In [150]:
import numpy as np
from transitions import Machine

STEP_SIZE = 10
# Note: cooling is positive in GHz and heating is negative in GHz

NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 50  # ± this number in GHz

MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
MRM_FREQ_UNCERTAINTY = 100  # in GHz
INITIAL_COOLDOWN = 200 # in GHz -- should clear laser uncertainty and mrm uncertainty

WHICH_RESONANCES = list(range(-10, 10, 1))  # will track the resonances -10*FSR over, ..., 0*FSR over, ... 9*FSR over
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to consider upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)        

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)
        
        laser_lines += self._offset    
    
        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY
            else:
                delta = np.random.uniform(-1*LASER_FREQ_UNCERTAINTY,
                                          LASER_FREQ_UNCERTAINTY,
                                          laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring):
        assert self._brightness[which_laser] == 'bright', \
            f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"
    
    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):

    states = ['idle', 'cold', 'search', 'lock']

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):
        # jump to the laser signal we want to capture
        self.loc = self.laser_locs_seen[-2]
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    if self._previous_ring is None:
                        this_ring = 0
                    else:
                        this_ring = self._previous_ring + 1
                    self._lasers.capture_laser_line(idx, this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        # In this case, we consider the simple algo.
        # if we have seen one dim and one bright, we have found all lines
        if self._previous_ring is None:  # this is the first ring
            if len(self.laser_locs_seen) >= 3:
                self._found_all_lines = True
        else:  # all the other rings
            if len(self.laser_locs_seen) >= 3:
                if self.laser_brightness_seen[-3] != 'bright' \
                        and self.laser_brightness_seen[-2] == 'bright':
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

In [151]:
from collections import deque
import copy


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):

    mrm_lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * MRM_FREQ_SPACING
    mrm_lines = np.vstack([mrm_lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * MRM_FSR
    mrm_lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= MRM_FREQ_UNCERTAINTY
        else:
            delta = np.random.uniform(-1*MRM_FREQ_UNCERTAINTY,
                                      MRM_FREQ_UNCERTAINTY,
                                      NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        mrm_lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-1*MRM_FSR, MRM_FSR)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        mrm_lines += global_delta

    return mrm_lines


def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent
    
    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent

In [154]:
# Try out the algorithm with a single try
lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING, variations=True, corners=True)
all_ring_resonances = populate_mrm_resonances(variations=True, corners=True)

all_rings = []

for idx in range(NUM_OF_LASERS):
    if idx == 0:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers)
    else:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers, previous_ring=idx-1)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_rings.append(mrm)

    print(f"Locked to laser # {mrm.locked_laser_idx}")
    print(lasers.brightness)

lasers_locked_to = [int(m.locked_laser_idx) for m in all_rings]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(lasers_locked_to, BARREL_ROLL_SEQ))

amt_heat_applied = [m.loc for m in all_rings]
print(f"Amount of heat applied: {amt_heat_applied}")

Locked to laser # 4
['bright', 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 0
[1, 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 5
[1, 'bright', 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 1
[1, 3, 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 6
[1, 3, 'bright', 'bright', 0, 2, 4, 'bright']
Locked to laser # 2
[1, 3, 5, 'bright', 0, 2, 4, 'bright']
Locked to laser # 7
[1, 3, 5, 'bright', 0, 2, 4, 6]
Locked to laser # 3
[1, 3, 5, 7, 0, 2, 4, 6]
Barrel roll check? True
Amount of heat applied: [-250, -450, -550, -250, -450, -250, -250, -450]


In [166]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy

eval_corners = False

# With cycling:
cycling_freqs = [-k * 50 for k in range(18)]  # change in frequency due to cycling
# cycling_freqs = [-k * 100 for k in range(9)]  # change in frequency due to cycling
# Without cycling: uncomment the one below for no cycling
# cycling_freqs = [0]

num_trials = 50
trial_results = []

for _ in range(num_trials):
    lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING, variations=True, corners=eval_corners)
    all_ring_resonances = populate_mrm_resonances(global_variations=True,
                                                  variations=True,
                                                  corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):

        lasers_ = copy.deepcopy(lasers)
        all_rings = []

        for idx in range(NUM_OF_LASERS):
            if idx == 0:  # the first ring
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq)
            else:
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq,
                              previous_ring=idx-1)
            # Run the search algorithm for each MRM
            mrm.start()

            while not mrm.found_all_lines:
                mrm.keep_searching()

            mrm.lock_to_line()
            all_rings.append(mrm)

        laser_sequence = [int(m.locked_laser_idx) for m in all_rings]
        amt_heat_applied = [m.loc for m in all_rings]
        colder = [n for n in amt_heat_applied if n >= 0]
        hotter = [n for n in amt_heat_applied if n <= 0]

        max_cool = max(colder) if len(colder) > 0 else 0
        max_heat = min(hotter) if len(hotter) > 0 else 0
        
        max_diff = max_cool - max_heat

        # if True, cycle; else, success and no need for cycling
        cycling_condition = abs(max_heat) > (MRM_FSR * .5)
        if not cycling_condition:
            break

    result = {'laser_sequence': laser_sequence,
              'barrel_roll': check_barrel_roll(laser_sequence, BARREL_ROLL_SEQ), # check if barrel roll equivalent
              'max_cool': max_cool,
              'max_heat': max_heat,
              'max_diff': max_diff,
              'cycling_trial': ctry,
              'successful_lock': not cycling_condition  # check if we were able to lock with < 1/2 * FSR of heat
              }
    
    ring_numbers = list(range(NUM_OF_LASERS))
    heat_result = dict(zip(ring_numbers, amt_heat_applied))  # (-) value is heat and (+) value is cool
    result.update(heat_result)

    trial_results.append(result)

trial_results = pd.DataFrame(trial_results)



In [167]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Heat and cool amount
fig = px.histogram(np.array(trial_results[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 100)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

# Plot the maximum differences
fig = px.histogram(trial_results, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 100)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

In [164]:
# check the results: barrel-roll and meeting our heat threshold requirements

print("All in barrel-roll sequence? ", trial_results['barrel_roll'].all())
print("All locked with 1/2 * FSR condition?", trial_results['successful_lock'].all())


All in barrel-roll sequence?  True
All locked with 1/2 * FSR condition? False


In [165]:
# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown(trial_results.to_markdown()))

|    | laser_sequence           | barrel_roll   |   max_cool |   max_heat |   max_diff |   cycling_trial | successful_lock   |    0 |    1 |     2 |    3 |    4 |     5 |    6 |    7 |
|---:|:-------------------------|:--------------|-----------:|-----------:|-----------:|----------------:|:------------------|-----:|-----:|------:|-----:|-----:|------:|-----:|-----:|
|  0 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -320 |        330 |               0 | True              | -110 |  -50 |   -30 | -220 |  -20 |    10 |  -50 | -320 |
|  1 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -560 |        560 |               0 | True              | -270 | -420 |  -340 | -430 | -350 |  -560 | -330 | -450 |
|  2 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -350 |        350 |               0 | True              | -220 |  -80 |  -270 | -220 | -270 |  -170 | -350 | -310 |
|  3 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -530 |        530 |               1 | True              | -220 | -240 |  -340 | -320 | -310 |  -270 | -350 | -530 |
|  4 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -470 |        470 |               0 | True              | -390 | -390 |  -310 | -330 | -320 |  -400 | -400 | -470 |
|  5 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -510 |        510 |               0 | True              | -130 | -120 |  -200 | -180 | -150 |  -370 | -300 | -510 |
|  6 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -420 |        420 |               0 | True              |  -70 | -420 |  -210 | -260 | -270 |  -220 | -210 | -280 |
|  7 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |      -2720 |       2720 |               8 | False             | -860 | -800 | -2720 | -960 | -960 | -1100 | -880 | -980 |
|  8 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -490 |        490 |               2 | True              | -320 | -340 |  -350 | -260 | -490 |  -320 | -470 | -470 |
|  9 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -420 |        420 |               0 | True              | -120 | -180 |  -420 | -370 | -290 |  -350 | -320 | -360 |
| 10 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -400 |        400 |               0 | True              |  -20 | -250 |  -280 | -370 | -320 |  -400 | -300 | -230 |
| 11 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -520 |        520 |               1 | True              | -200 | -260 |  -490 | -520 | -500 |  -480 | -410 | -360 |
| 12 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -340 |        340 |               1 | True              | -190 | -310 |  -280 | -280 | -310 |  -250 | -340 | -240 |
| 13 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -450 |        450 |               2 | True              | -300 | -280 |  -210 | -330 | -360 |  -230 | -450 | -360 |
| 14 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -460 |        460 |               0 | True              | -190 | -300 |  -240 | -150 | -270 |  -460 | -420 | -430 |
| 15 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -440 |        440 |               0 | True              | -150 | -130 |  -440 | -290 | -360 |  -380 | -300 | -430 |
| 16 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -790 |        790 |               4 | True              | -590 | -460 |  -780 | -720 | -760 |  -790 | -680 | -640 |
| 17 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -630 |        630 |               1 | True              | -250 | -320 |  -310 | -380 | -260 |  -410 | -580 | -630 |
| 18 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -420 |        420 |               0 | True              | -200 |  -90 |  -220 | -210 | -220 |  -420 | -290 | -340 |
| 19 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -570 |        570 |               1 | True              | -220 | -150 |  -150 | -520 | -450 |  -360 | -530 | -570 |
| 20 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -520 |        520 |               0 | True              | -180 | -160 |   -60 | -120 | -240 |  -160 | -520 | -410 |
| 21 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -510 |        510 |               0 | True              | -140 | -260 |  -230 | -480 | -380 |  -430 | -510 | -360 |
| 22 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -540 |        540 |               0 | True              | -360 | -250 |  -370 | -370 | -330 |  -380 | -430 | -540 |
| 23 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -470 |        470 |               0 | True              | -210 | -190 |  -130 | -160 | -470 |  -450 | -460 | -470 |
| 24 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -350 |        350 |               0 | True              |  -40 | -180 |   -40 | -130 | -110 |  -120 | -350 | -240 |
| 25 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -380 |        380 |               0 | True              | -370 | -380 |  -240 | -250 | -340 |  -310 | -230 | -280 |
| 26 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -570 |        570 |               0 | True              | -340 | -320 |  -370 | -230 | -350 |  -330 | -400 | -570 |
| 27 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -480 |        480 |               0 | True              |  -60 | -330 |  -460 | -480 | -360 |  -290 | -340 | -380 |
| 28 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -430 |        430 |               1 | True              | -240 | -160 |  -340 | -430 | -420 |  -360 | -390 | -430 |
| 29 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -370 |        370 |               0 | True              | -130 | -340 |  -240 | -310 | -370 |  -210 | -330 | -280 |
| 30 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -410 |        410 |               0 | True              | -160 | -220 |  -180 | -270 | -380 |  -400 | -410 | -330 |
| 31 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -460 |        460 |               0 | True              | -180 |  -80 |  -290 | -390 | -390 |  -460 | -320 | -280 |
| 32 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -410 |        410 |               0 | True              | -210 | -410 |  -280 | -330 | -360 |  -240 | -350 | -370 |
| 33 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -440 |        440 |               0 | True              | -130 | -250 |   -50 | -250 | -440 |  -400 | -440 | -290 |
| 34 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 |               0 | True              | -290 | -230 |  -100 | -220 | -130 |  -260 |  -80 | -370 |
| 35 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -410 |        410 |               0 | True              | -320 | -330 |  -370 | -300 | -210 |  -240 | -230 | -410 |
| 36 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -630 |        630 |               4 | True              | -570 | -460 |  -460 | -620 | -600 |  -630 | -540 | -550 |
| 37 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -570 |        570 |               2 | True              | -570 | -430 |  -370 | -400 | -520 |  -500 | -470 | -390 |
| 38 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -370 |        380 |               0 | True              | -170 | -130 |    10 | -110 |  -10 |   -40 | -300 | -370 |
| 39 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -380 |        380 |               0 | True              | -230 | -230 |  -240 | -290 | -380 |  -180 | -170 | -230 |
| 40 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -450 |        450 |               0 | True              |  -80 | -300 |  -360 | -240 | -370 |  -450 | -310 | -440 |
| 41 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -420 |        420 |               0 | True              | -420 | -370 |  -370 | -250 | -360 |  -300 | -390 | -260 |
| 42 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -500 |        500 |               0 | True              | -160 | -300 |  -450 | -320 | -300 |  -500 | -460 | -440 |
| 43 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -410 |        420 |               0 | True              |  -10 | -120 |    10 |  -70 | -160 |  -290 | -260 | -410 |
| 44 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -300 |        300 |               0 | True              | -210 | -230 |  -190 | -200 | -300 |  -280 | -300 | -300 |
| 45 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -530 |        530 |               0 | True              | -170 | -250 |  -220 | -320 | -490 |  -530 | -380 | -500 |
| 46 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -580 |        580 |               2 | True              | -300 | -260 |  -220 | -580 | -560 |  -380 | -440 | -440 |
| 47 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -370 |        370 |               0 | True              |  -40 | -100 |  -150 | -320 | -360 |  -350 | -370 | -190 |
| 48 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -490 |        490 |               0 | True              | -170 | -220 |  -280 | -120 | -480 |  -400 | -430 | -490 |
| 49 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -610 |        610 |               1 | True              | -350 | -190 |  -300 | -330 | -610 |  -450 | -420 | -490 |